In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential, load_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

In [2]:
df = pd.read_csv('/content/Housing.csv')

In [3]:
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [4]:
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()

In [5]:
df['mainroad'] = Encoder.fit_transform(df['mainroad'])
df['guestroom'] = Encoder.fit_transform(df['guestroom'])
df['basement'] = Encoder.fit_transform(df['basement'])
df['hotwaterheating'] = Encoder.fit_transform(df['hotwaterheating'])
df['airconditioning'] = Encoder.fit_transform(df['airconditioning'])
df['prefarea'] = Encoder.fit_transform(df['prefarea'])
df['furnishingstatus'] = Encoder.fit_transform(df['furnishingstatus'])

In [6]:
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,0
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,0
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,0


In [7]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
model = Sequential()
model.add(Dense(18, activation='swish', input_shape=(X_train.shape[1],)))
model.add(Dense(26, activation='swish'))
model.add(Dense(20, activation='swish'))
model.add(Dense(15, activation='swish'))
model.add(Dense(1))

model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
history = model.fit(X_train, y_train, epochs=200, batch_size=64, validation_split=0.1)

Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 126ms/step - loss: 1.4096 - mae: 0.9839 - val_loss: 1.4398 - val_mae: 1.0724
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.1936 - mae: 0.9284 - val_loss: 1.1612 - val_mae: 0.9599
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.9481 - mae: 0.8392 - val_loss: 0.9598 - val_mae: 0.8649
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.8511 - mae: 0.7958 - val_loss: 0.7726 - val_mae: 0.7583
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.7041 - mae: 0.7207 - val_loss: 0.6296 - val_mae: 0.6754
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.6217 - mae: 0.6716 - val_loss: 0.5351 - val_mae: 0.6076
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.5575 - mae: 0.6220 - val_loss: 0.5097 - val_mae: 0.5884
Epoch 8/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.5535 - mae: 0.6166 - val_loss: 0.5095 - val_mae: 0.5873
Epoch 9/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.5363 - mae: 0.6063 - va

In [12]:
train_loss,train_mae = model.evaluate(X_train, y_train, verbose=0)
test_loss,test_mae = model.evaluate(X_test, y_test, verbose=0)

print(f"Training Loss: {train_loss}, Training MAE: {train_mae}")
print(f"Testing Loss: {test_loss}, Testing MAE: {test_mae}")

Training Loss: 0.34200865030288696, Training MAE: 0.4555715024471283
Testing Loss: 0.7098841667175293, Testing MAE: 0.6839146614074707


In [14]:
model.save('housing_price_model.h5')

In [18]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
def mse(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_pred - y_true))

loaded_model = load_model('housing_price_model.h5', custom_objects={'mse': mse})

new_data = pd.DataFrame([{
    'area': 1500,
    'bedrooms': 3,
    'bathrooms': 2,
    'stories': 2,
    'mainroad': 1,
    'guestroom': 1,
    'basement': 1,
    'hotwaterheating': 1,
    'airconditioning': 1,
    'parking': 2,
    'prefarea': 1,
    'furnishingstatus': 1
}])

scaler = StandardScaler()

scaler.fit(X_train)

new_data_scaled = scaler.transform(new_data)

prediction = loaded_model.predict(new_data_scaled)

print(f'Predicted Housing Price: {prediction[0][0]}')


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
Predicted Housing Price: 622.6371459960938
